### Настройка Airflow

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься настройкой среды исполнения, а сразу начать писать код и работать с Airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

#### Запуск шедулера и вебсервиса

In [ ]:
# Запуск шедулера
!airflow scheduler -D

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/


In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken <YOUR TOKEN> # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

nohup: redirecting stderr to stdout


После запуска команды выше, перейдите по адресу в ngrok и подождите  пока появится DAG с именем dag

### Практика

#### Автогенерация задач

Поместите в dag.py следующий код. Посмотрите результаты в веб интерфейсе.

**Задание #1**

Сложите все номера задач, это будет ответом.

```python
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))

tasks_list=list()
for i in range(0, 10):
    tasks_list.append(DummyOperator(task_id=f'task_{i}', dag=dag))
    if i:
      tasks_list[i-1] >> tasks_list[i]
```

In [ ]:
sum(range(0,10))

#### Автогенерация дагов

Поместите в dag.py следующий код. Посмотрите результаты в веб интерфейсе.

**Задание #2**

Сложите сумму дагов друг с другом, это и будет ответ.

```python
from airflow import DAG
from datetime import datetime
from airflow.operators.dummy_operator import DummyOperator

def create_dag(dag_id,
               default_args,
               schedule='@daily'):

    dag = DAG(dag_id,
              schedule_interval=schedule,
              default_args=default_args)

    with dag:
        t1 = DummyOperator(task_id=f'task', dag=dag)

    return dag


# build a dag for each number in range(10)
for n in range(1, 4):
    dag_id = f'dag_{n+65}'

    default_args = {'owner': 'airflow',
                    'start_date': datetime(2021, 1, 1)
                    }


    globals()[dag_id] = create_dag(dag_id, default_args)
```